In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler



def count(df):
    counts = df.groupby(['label', 'hour']).size().reset_index(name='count')
    df = df.merge(counts, on=['label', 'hour'], how='left')
    return df


#open weather data 2018, read textfile and convert to dataframe
text = open('data/2018_weather_data.txt', 'r')
weather_data = text.read()
weather_data = weather_data.split('\n')
weather_data = [i.split('\t') for i in weather_data[1:]]
weather_data = pd.DataFrame(weather_data)
weather_data.columns = ['Date', 'Max_temp', 'Min_temp', 'Avg_temp', 'Departure_temp', 'HDD', 'CDD', 'Precipitation', 'Snowfall', 'Snow_depth']
weather_data = weather_data.dropna()
# day month
weather_data['Date'] = pd.to_datetime(weather_data['Date'])
weather_data['day'] = weather_data['Date'].dt.day
weather_data['month'] = weather_data['Date'].dt.month
weather_data['weekend'] = np.where(weather_data['Date'].dt.dayofweek < 5, 0, 1)
weather_data['day_of_week'] = weather_data['Date'].dt.dayofweek
weather_data['year'] = weather_data['Date'].dt.year
weather_data.drop(['Date'], axis=1, inplace=True)

weather_data.replace('T', 0, inplace=True)
weather_data.replace('M', 0, inplace=True)

# convert temperatures to celcius
weather_data['Max_temp'] = (weather_data['Max_temp'].astype(float) - 32) * 5/9
weather_data['Min_temp'] = (weather_data['Min_temp'].astype(float) - 32) * 5/9
weather_data['Avg_temp'] = (weather_data['Avg_temp'].astype(float) - 32) * 5/9
weather_data['Departure_temp'] = (weather_data['Departure_temp'].astype(float) - 32) * 5/9


# check for nan
print(weather_data.isnull().sum())

print(weather_data.head())

df = pd.read_csv('data/Trips_2018.csv')
df.rename(columns={'Unnamed: 0': 'trip_id'}, inplace=True)
df.set_index('trip_id', inplace=True)
df = df.dropna()
df['starttime'] = pd.to_datetime(df['starttime'], format="%Y-%m-%d %H:%M:%S.%f")
df['stoptime'] = pd.to_datetime(df['stoptime'], format="%Y-%m-%d %H:%M:%S.%f")
df = df[~np.isnan(df['start_station_id'])]
df = df[~np.isnan(df['end_station_id'])]
# get rid of Canada outlier
df = df[df['start_station_longitude'] < -73.6]
df = df[df['end_station_longitude'] < -73.6]
df = pd.get_dummies(df, columns=['usertype'], dtype=int, drop_first=True)

print(df.columns)
print(df.head())

df_arrival = df.drop(['start_station_latitude', 'start_station_longitude', 'start_station_id', 'starttime'], axis=1)
df_departure = df.drop(['end_station_latitude', 'end_station_longitude', 'end_station_id', 'stoptime'], axis=1)

# make lat and long called that and time
df_arrival.rename(columns={'end_station_latitude': 'latitude', 'end_station_longitude': 'longitude'}, inplace=True)
df_departure.rename(columns={'start_station_latitude': 'latitude', 'start_station_longitude': 'longitude'}, inplace=True)
df_arrival.rename(columns={'stoptime': 'time'}, inplace=True)
df_departure.rename(columns={'starttime': 'time'}, inplace=True)

print(df_arrival.columns)
print(df_departure.columns)

# add hour, day, month, weekend, day of week
df_arrival['hour'] = df_arrival['time'].dt.hour
df_departure['hour'] = df_departure['time'].dt.hour
df_arrival['day'] = df_arrival['time'].dt.day
df_departure['day'] = df_departure['time'].dt.day
df_arrival['month'] = df_arrival['time'].dt.month
df_departure['month'] = df_departure['time'].dt.month
df_arrival['weekend'] = np.where(df_arrival['time'].dt.dayofweek < 5, 0, 1)
df_departure['weekend'] = np.where(df_departure['time'].dt.dayofweek < 5, 0, 1)
df_arrival['day_of_week'] = df_arrival['time'].dt.dayofweek
df_departure['day_of_week'] = df_departure['time'].dt.dayofweek

kmeans = KMeans(n_clusters=20, random_state=0, n_init='auto').fit(df_arrival[['latitude', 'longitude']])

# add labels using k means predict
df_arrival['label'] = kmeans.predict(df_arrival[['latitude', 'longitude']])
df_departure['label'] = kmeans.predict(df_departure[['latitude', 'longitude']])




Max_temp          0
Min_temp          0
Avg_temp          0
Departure_temp    0
HDD               0
CDD               0
Precipitation     0
Snowfall          0
Snow_depth        0
day               0
month             0
weekend           0
day_of_week       0
year              0
dtype: int64
  Max_temp Min_temp Avg_temp Departure_temp HDD CDD Precipitation Snowfall  \
0       19        7     13.0          -22.2  52   0          0.00      0.0   
1       26       13     19.5          -15.5  45   0          0.00      0.0   
2       30       16     23.0          -11.8  42   0          0.00      0.0   
3       29       19     24.0          -10.7  41   0          0.76      9.8   
4       19        9     14.0          -20.5  51   0          0.00      0.0   

  Snow_depth  day  month  weekend  day_of_week  year  
0          0    1      1        0            0  2018  
1          0    2      1        0            1  2018  
2          0    3      1        0            2  2018  
3          1    4 

In [2]:
# count
df_arrival = count(df_arrival)
df_departure = count(df_departure)
print('added count')

added count


In [3]:
# drop duplicates
df_arrival = df_arrival.drop_duplicates()
df_departure = df_departure.drop_duplicates()


print(df_arrival.head())
print(df_departure.head())


   tripduration                    time  end_station_id   latitude  longitude  \
0           970 2018-01-01 14:07:08.186           505.0  40.749013 -73.988484   
1           723 2018-01-01 15:45:33.341          3255.0  40.750585 -73.994685   
2           496 2018-01-01 15:47:35.172           525.0  40.755942 -74.002116   
3           306 2018-01-01 15:45:20.191           447.0  40.763707 -73.985162   
4           306 2018-01-01 18:19:57.642          3356.0  40.774667 -73.984706   

   bikeid  birth_year  gender  usertype_Subscriber  hour  day  month  weekend  \
0   31956        1992       1                    1    14    1      1        0   
1   32536        1969       1                    1    15    1      1        0   
2   16069        1956       1                    1    15    1      1        0   
3   31781        1974       1                    1    15    1      1        0   
4   30319        1992       1                    1    18    1      1        0   

   day_of_week  label   co

In [4]:
df_departure.columns

Index(['tripduration', 'time', 'start_station_id', 'latitude', 'longitude',
       'bikeid', 'birth_year', 'gender', 'usertype_Subscriber', 'hour', 'day',
       'month', 'weekend', 'day_of_week', 'label', 'count'],
      dtype='object')

# Visualizations and data exploration
Firtstly let's examine the the weather data we included and establish which witch features are contributing to the most variance in the data. We will also look at the distribution of the data to see if there are any outliers or other anomalies that we should be aware of.


In [45]:
# make a temperature average plot for each month
weather_data_per_month = weather_data[['month', 'Avg_temp']]

print(weather_data_per_month.head(15))
#calculate average temperature per month
weather_data_per_month = weather_data_per_month.groupby(['month'], axis=1)





    month Avg_temp
0       1     13.0
1       1     19.5
2       1     23.0
3       1     24.0
4       1     14.0
5       1      9.5
6       1     11.5
7       1     24.0
8       1     37.0
9       1     36.5
10      1     47.0
11      1     52.5
12      1     38.5
13      1     20.0
14      1     23.0


/var/folders/9z/rxmxhrbj6n38vk6t9797bdpw0000gp/T/ipykernel_54511/1039918086.py:6: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  weather_data_per_month = weather_data_per_month.groupby(['month'], axis=1)


In [9]:
# # drop column latitiude and longitude, bikeid, birth_year, gender, user type

# df_arrival.drop(['end_station_id', 'latitude', 'longitude',
#        'bikeid', 'birth_year', 'gender', 'usertype_Subscriber', 
#        'month'], inplace=True, axis=1)



In [14]:
# df_arrival.sort_values(by=['label'], inplace=True)
# print(df_arrival.head())
# print(df_arrival.columns)
# print(df_arrival.shape)

         tripduration                    time  hour  day  weekend  \
115400            196 2018-01-01 00:05:07.438     0    1        0   
2765191           370 2018-04-17 07:25:14.507     7   17        0   
9266230          1470 2018-07-30 11:43:48.602    11   30        0   
8034449          1003 2018-07-30 11:43:48.585    11   30        0   
7849307          1817 2018-07-30 11:43:30.760    11   30        0   

         day_of_week  label  count  
115400             0      0   8341  
2765191            1      0  66700  
9266230            0      0  56027  
8034449            0      0  56027  
7849307            0      0  56027  
Index(['tripduration', 'time', 'hour', 'day', 'weekend', 'day_of_week',
       'label', 'count'],
      dtype='object')
(17545760, 8)
